In [1]:
import pandas as pd
import os
import numpy as np
import xlsxwriter as xw
import re
import warnings
warnings.filterwarnings('ignore')
import openpyxl as op
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from pathlib import Path

# Fill the Ingestion Date and Revisit Expire Date

In [2]:
Ingestion_Date = "11-Nov-2022"
Default_Ingestion_Date = "8-Jun-2022"
Revisit_Expire_Date = "11-Nov-2022"

# Target file

# Extracting the latest file

In [3]:
input_loc = '/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 1 Invalid file read for dynamics/Target/Dynamic/'
output_loc = '/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 1 Invalid file read for dynamics/Result/'

In [4]:
filelist = os.listdir(input_loc)

latest_dataset = pd.DataFrame()

for file in filelist:
    if file.endswith('.xlsx'):
        df_1 = pd.read_excel(input_loc+file)
        latest_dataset = latest_dataset.append(df_1)
    elif file.endswith('csv'):
        df_2 = pd.read_csv(input_loc+file)
        latest_dataset = latest_dataset.append(df_2)

In [5]:
latest_dataset.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,RWM Remarks,Ingestion Date,RWM Revisit Status,RWM Revisit Date,Revisit Expire Date,Validity,Pincode,Phone Number,City,Cluster,Failed Upload RWM MID,csv Google MID,Google csv Visit Date,csv Validity,s no
0,BCR2DN4TSCO3DNDR,checkin-nob,ADD,5-Aug-2022,Success,5-Aug-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [6]:
latest_dataset.sort_values(by=['s no'], inplace=True)

In [7]:
latest_dataset_filter = latest_dataset[latest_dataset['Report Date'].isin([Ingestion_Date])]

In [8]:
latest_dataset_filter.shape

(133036, 19)

In [9]:
latest_dataset_f = latest_dataset_filter[['MerchantId', 'Objectives', 'IngestionStatus', 'Report Date', 's no']]

In [10]:
latest_dataset_f['Primary_key'] = range(1,(len(latest_dataset_f)+1))

In [11]:
latest_dataset_f.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,s no,Primary_key
0,BCR2DN4T7SL2PPY6,checkin-nob,ADD,11-Nov-2022,2278168,1


In [12]:
old_dataset = latest_dataset[~latest_dataset["Report Date"].isin([Ingestion_Date])]

In [13]:
old_dataset.shape

(1991413, 19)

# Change file name

In [14]:
Break =

SyntaxError: invalid syntax (816266325.py, line 1)

# Extracting the file from the invalid to match

In [15]:
input_loc_invalid = '/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 1 Invalid file read for dynamics/Target/Invalid/'
output_loc_invalid = '/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 1 Invalid file read for dynamics/Result/'

#### Extracting Delete

In [19]:
failed_delete = pd.read_excel(input_loc_invalid+'Invalid_delete_consolidated.xlsx')

In [20]:
failed_delete_f = failed_delete[['Merchant Ext.ID', 'Error']].rename(columns={'Error':'Remarks_Delete'})

In [21]:
failed_delete_f.head(1)

,Merchant Ext.ID,Remarks_Delete
0,Empty,Empty


#### Extracting Adhoc

In [22]:
failed_edit = pd.read_excel(input_loc_invalid+'Invalid_adhoc_consolidated.xlsx')

In [23]:
failed_edit_f = failed_edit[['Merchant Ext.ID', 'Error']].rename(columns={'Error':'Remarks_Edit'})

In [24]:
failed_edit_f.head(1)

,Merchant Ext.ID,Remarks_Edit
0,BCR2DN4T7SL2PPY6,Merchant Ext. ID is not present in the Revisit...


#### Extracting optional

In [25]:
failed_add = pd.read_excel(input_loc_invalid+'Invalid_optional_consolidated.xlsx')

In [26]:
failed_add_f = failed_add[['Merchant Ext.ID', 'Remarks']].rename(columns={'Remarks':'Remarks_Add'})

In [27]:
failed_add_f.head(1)

,Merchant Ext.ID,Remarks_Add
0,BCR2DN4T7SL2PPY6,Merchant Ext.ID is not present in Merchant Mas...


# Matching the Delete, Adhoc and optional one by one to the latest dynamic dataset.
# Fill the status of the latest dataset

In [28]:
latest_dataset_f['Status'] = np.nan

In [29]:
latest_dataset_f.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,s no,Primary_key,Status
0,BCR2DN4T7SL2PPY6,checkin-nob,ADD,11-Nov-2022,2278168,1,NaN


## Delete

#### Filter the Delete from the latest dataset_f

In [30]:
latest_dataset_f.shape

(133036, 7)

In [31]:
latest_dataset_delete = latest_dataset_f[latest_dataset_f['IngestionStatus'].isin(['DELETE'])]

In [32]:
latest_dataset_delete.shape

(0, 7)

In [33]:
latest_dataset_delete.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,s no,Primary_key,Status


#### Merging the invalid delete with the latest_dataset and filling the status

In [34]:
failed_delete_f.head(1)

,Merchant Ext.ID,Remarks_Delete
0,Empty,Empty


In [35]:
failed_delete_f.rename(columns={'Merchant Ext.ID':'MerchantId'}, inplace = True)

In [36]:
latest_dataset_delete_merge = latest_dataset_delete.merge(failed_delete_f, how="left", on='MerchantId')

In [37]:
latest_dataset_delete_merge.drop_duplicates(subset=['Primary_key'], inplace=True)

,Objectives,IngestionStatus,Report Date,s no,Primary_key,Status,MerchantId,Remarks_Delete


In [38]:
latest_dataset_delete_merge['Status'] = latest_dataset_delete_merge['Remarks_Delete'].isnull()

In [39]:
latest_dataset_delete_merge['Status'] = latest_dataset_delete_merge['Status'].map({True:'Delete', False:'Failed'})

In [40]:
latest_dataset_delete_merge_f = latest_dataset_delete_merge[[
    'MerchantId', 'Status', 'Remarks_Delete']].rename(columns={'Status':'Status_Delete'})

In [41]:
latest_dataset_delete_merge_f.head(1)

,MerchantId,Status_Delete,Remarks_Delete


## Edit

#### Filter the Delete from the latest dataset_f

In [42]:
latest_dataset_edit = latest_dataset_f[latest_dataset_f['IngestionStatus'].isin(['EDIT'])]

In [43]:
latest_dataset_edit.shape

(27063, 7)

In [44]:
latest_dataset_edit.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,s no,Primary_key,Status
3,BCR2DN4TUDZ4XNLL,Inactive - INR 40 Extra Pay on Txn,EDIT,11-Nov-2022,2278171,4,NaN


#### Merging the latest_dataset_edit with the invalid adhoc and fill the status

In [45]:
failed_edit_f.head(1)

,Merchant Ext.ID,Remarks_Edit
0,BCR2DN4T7SL2PPY6,Merchant Ext. ID is not present in the Revisit...


In [46]:
failed_edit_f.rename(columns={'Merchant Ext.ID':'MerchantId'}, inplace = True)

In [47]:
latest_dataset_edit_merge = latest_dataset_edit.merge(failed_edit_f, how='left', on='MerchantId')

In [48]:
latest_dataset_edit_merge.drop_duplicates(subset=['Primary_key'], inplace=True)

In [49]:
latest_dataset_edit_merge.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,s no,Primary_key,Status,Remarks_Edit
0,BCR2DN4TUDZ4XNLL,Inactive - INR 40 Extra Pay on Txn,EDIT,11-Nov-2022,2278171,4,NaN,Merchant Ext. ID is not present in the Revisit...


In [50]:
latest_dataset_edit_merge['Status'] = latest_dataset_edit_merge['Remarks_Edit'].isnull()
latest_dataset_edit_merge['Status'] = latest_dataset_edit_merge['Status'].map({True:'Ongoing', False:'Failed'})

In [51]:
latest_dataset_edit_merge.rename(columns={'Status':'Status_Edit'}, inplace = True)

In [52]:
latest_dataset_edit_merge.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,s no,Primary_key,Status_Edit,Remarks_Edit
0,BCR2DN4TUDZ4XNLL,Inactive - INR 40 Extra Pay on Txn,EDIT,11-Nov-2022,2278171,4,Failed,Merchant Ext. ID is not present in the Revisit...


In [53]:
latest_dataset_edit_merge_f = latest_dataset_edit_merge[['MerchantId', 'Status_Edit', 'Remarks_Edit']]

In [54]:
latest_dataset_edit_merge_f.head(1)

,MerchantId,Status_Edit,Remarks_Edit
0,BCR2DN4TUDZ4XNLL,Failed,Merchant Ext. ID is not present in the Revisit...


# Adding

#### Filter the Add from the latest dataset_f

In [55]:
latest_dataset_f.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,s no,Primary_key,Status
0,BCR2DN4T7SL2PPY6,checkin-nob,ADD,11-Nov-2022,2278168,1,NaN


In [56]:
latest_dataset_add = latest_dataset_f[latest_dataset_f['IngestionStatus'].isin(['ADD'])]

In [57]:
latest_dataset_add.shape

(105973, 7)

#### Merging the latest_dataset_add with the invalid Optional and fill the status

In [58]:
failed_add_f.head(1)

,Merchant Ext.ID,Remarks_Add
0,BCR2DN4T7SL2PPY6,Merchant Ext.ID is not present in Merchant Mas...


In [59]:
failed_add_f.rename(columns={'Merchant Ext.ID':'MerchantId'}, inplace = True)

In [60]:
latest_dataset_add_merge = latest_dataset_add.merge(failed_add_f, how = 'left', on = 'MerchantId')

In [61]:
latest_dataset_add_merge.drop_duplicates(subset=['Primary_key'], inplace=True)

In [62]:
latest_dataset_add_merge['Status'] = latest_dataset_add_merge['Remarks_Add'].isnull()

In [63]:
latest_dataset_add_merge['Status'] = latest_dataset_add_merge['Status'].map({True:'Ongoing', False:'Failed'})

In [64]:
latest_dataset_add_merge_f = latest_dataset_add_merge[['MerchantId', 'Status', 'Remarks_Add']].rename(
    columns={'Status':'Status_add'})

In [65]:
latest_dataset_add_merge_f.head(1)

,MerchantId,Status_add,Remarks_Add
0,BCR2DN4T7SL2PPY6,Failed,Merchant Ext.ID is not present in Merchant Mas...


# Merging all the dataframe (delete, edit, and add) back in the latest dataframe
# And Updating status

In [66]:
latest_dataset_f.head()

,MerchantId,Objectives,IngestionStatus,Report Date,s no,Primary_key,Status
0,BCR2DN4T7SL2PPY6,checkin-nob,ADD,11-Nov-2022,2278168,1,NaN
1,BCR2DN4TUC7YFOJZ,checkin-nob,ADD,11-Nov-2022,2278169,2,NaN
2,BCR2DN4T4DFYN3SG,Inactive - INR 40 Extra Pay on Txn,ADD,11-Nov-2022,2278170,3,NaN
3,BCR2DN4TUDZ4XNLL,Inactive - INR 40 Extra Pay on Txn,EDIT,11-Nov-2022,2278171,4,NaN
4,BCR2DN4TWCA7BVCW,checkin-nob,ADD,11-Nov-2022,2278172,5,NaN


In [67]:
latest_dataset_delete_merge_f.head(1)

,MerchantId,Status_Delete,Remarks_Delete


In [68]:
latest_dataset_edit_merge_f.head(1)

,MerchantId,Status_Edit,Remarks_Edit
0,BCR2DN4TUDZ4XNLL,Failed,Merchant Ext. ID is not present in the Revisit...


In [69]:
latest_dataset_add_merge_f.head(1)

,MerchantId,Status_add,Remarks_Add
0,BCR2DN4T7SL2PPY6,Failed,Merchant Ext.ID is not present in Merchant Mas...


In [70]:
latest_dataset_merge_d = latest_dataset_f.merge(latest_dataset_delete_merge_f, how = 'left', on = 'MerchantId')

In [71]:
latest_dataset_merge_d.drop_duplicates(subset=['Primary_key'], inplace=True)

In [72]:
latest_dataset_merge_d.shape

(133036, 9)

In [73]:
latest_dataset_merge_d_e = latest_dataset_merge_d.merge(
    latest_dataset_edit_merge_f, how = 'left', on = 'MerchantId')

In [74]:
latest_dataset_merge_d_e.drop_duplicates(subset=['Primary_key'], inplace=True)

In [75]:
latest_dataset_merge_d_e.shape

(133036, 11)

In [76]:
latest_dataset_merge_d_e_a = latest_dataset_merge_d_e.merge(
    latest_dataset_add_merge_f, how = 'left', on='MerchantId')

In [77]:
latest_dataset_merge_d_e_a.drop_duplicates(subset=['Primary_key'], inplace=True)

In [78]:
latest_dataset_merge_d_e_a.head(5)

,MerchantId,Objectives,IngestionStatus,Report Date,s no,Primary_key,Status,Status_Delete,Remarks_Delete,Status_Edit,Remarks_Edit,Status_add,Remarks_Add
0,BCR2DN4T7SL2PPY6,checkin-nob,ADD,11-Nov-2022,2278168,1,NaN,NaN,NaN,NaN,NaN,Failed,Merchant Ext.ID is not present in Merchant Mas...
1,BCR2DN4TUC7YFOJZ,checkin-nob,ADD,11-Nov-2022,2278169,2,NaN,NaN,NaN,NaN,NaN,Failed,Merchant Ext.ID is not present in Merchant Mas...
2,BCR2DN4T4DFYN3SG,Inactive - INR 40 Extra Pay on Txn,ADD,11-Nov-2022,2278170,3,NaN,NaN,NaN,NaN,NaN,Failed,Merchant Ext.ID is not present in Merchant Mas...
3,BCR2DN4TUDZ4XNLL,Inactive - INR 40 Extra Pay on Txn,EDIT,11-Nov-2022,2278171,4,NaN,NaN,NaN,Failed,Merchant Ext. ID is not present in the Revisit...,NaN,NaN
4,BCR2DN4TWCA7BVCW,checkin-nob,ADD,11-Nov-2022,2278172,5,NaN,NaN,NaN,NaN,NaN,Failed,Merchant Ext.ID is not present in Merchant Mas...


In [79]:
latest_dataset_merge_d_e_a['Status'].fillna(latest_dataset_merge_d_e_a['Status_Delete'], inplace=True)

In [80]:
latest_dataset_merge_d_e_a['Status'].fillna(latest_dataset_merge_d_e_a['Status_Edit'], inplace=True)

In [81]:
latest_dataset_merge_d_e_a['Status'].fillna(latest_dataset_merge_d_e_a['Status_add'], inplace=True)

In [82]:
latest_dataset_merge_d_e_a['Remarks'] = np.nan

In [83]:
latest_dataset_merge_d_e_a['Remarks'].fillna(latest_dataset_merge_d_e_a['Remarks_Delete'], inplace=True)

In [84]:
latest_dataset_merge_d_e_a['Remarks'].fillna(latest_dataset_merge_d_e_a['Remarks_Edit'], inplace=True)

In [85]:
latest_dataset_merge_d_e_a['Remarks'].fillna(latest_dataset_merge_d_e_a['Remarks_Add'], inplace=True)

In [86]:
latest_dataset_final = latest_dataset_merge_d_e_a[['MerchantId', 'Objectives','IngestionStatus', 'Report Date',
                                            'Remarks', 'Status',"s no"]].rename(columns={'Remarks':'RWM Remarks'})

In [87]:
latest_dataset_final['RWM Remarks'].fillna('Success', inplace = True)

# Filling the Ingestion Date and Revisit Expire Date

### Ingestion Date for the Add

In [88]:
latest_dataset_final['IngestionStatus_add'] = latest_dataset_final['IngestionStatus'].str.contains('ADD')

In [89]:
latest_dataset_final['RWM Remarks_success'] = latest_dataset_final['RWM Remarks'].str.contains('Success')

In [90]:
latest_dataset_final['RWM Remarks_success'].replace(False,"No", inplace=True)

In [91]:
latest_dataset_final['IngestionStatus_add_success'] = latest_dataset_final[
    'IngestionStatus_add'] == latest_dataset_final['RWM Remarks_success']

In [92]:
latest_dataset_final['IngestionStatus_add_success'] = latest_dataset_final[
    'IngestionStatus_add_success'].map({False:np.nan, True:Ingestion_Date})

In [93]:
latest_dataset_final.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,RWM Remarks,Status,s no,IngestionStatus_add,RWM Remarks_success,IngestionStatus_add_success
0,BCR2DN4T7SL2PPY6,checkin-nob,ADD,11-Nov-2022,Merchant Ext.ID is not present in Merchant Mas...,Failed,2278168,True,No,NaN


### Ingestion Date for the Edit

In [94]:
latest_dataset_final['IngestionStatus_edit'] = latest_dataset_final['IngestionStatus'].str.contains('EDIT')

In [95]:
latest_dataset_final['IngestionStatus_edit_success'] = latest_dataset_final[
    'IngestionStatus_edit'] == latest_dataset_final['RWM Remarks_success']

In [96]:
latest_dataset_final['IngestionStatus_edit_success'] = latest_dataset_final[
    'IngestionStatus_edit_success'].map({True:Default_Ingestion_Date, False:np.nan})

# Fill na of the ingestion date with the IngestionStatus_edit_success and IngestionStatus_add_success date

In [97]:
latest_dataset_final['Ingestion Date'] = np.nan

In [98]:
latest_dataset_final['Ingestion Date'].fillna(latest_dataset_final['IngestionStatus_add_success'],inplace=True)

In [99]:
latest_dataset_final['Ingestion Date'].fillna(latest_dataset_final['IngestionStatus_edit_success'],inplace=True)

### Revisit Expire Date for the Delete

In [100]:
latest_dataset_final['Revisit Expire Date'] = latest_dataset_final['Status'].str.contains("Delete")

In [101]:
latest_dataset_final['Revisit Expire Date'].replace(True, np.nan, inplace=True)

In [102]:
latest_dataset_final['Revisit Expire Date'].replace(np.nan, Revisit_Expire_Date , inplace=True)

In [103]:
latest_dataset_final['Revisit Expire Date'].replace(False, np.nan, inplace=True)

# Filter the columns and get the results

In [104]:
latest_dataset_final2 = latest_dataset_final[['MerchantId', 'Objectives', 'IngestionStatus', 'Report Date',
                                             'RWM Remarks', 'Ingestion Date', 'Revisit Expire Date', 's no']]

In [112]:
latest_dataset_final2.to_csv(output_loc+'latest_dataset_final.csv',index=False)

In [106]:
latest_dataset_final2.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,RWM Remarks,Ingestion Date,Revisit Expire Date,s no
0,BCR2DN4T7SL2PPY6,checkin-nob,ADD,11-Nov-2022,Merchant Ext.ID is not present in Merchant Mas...,NaN,NaN,2278168


In [107]:
latest_dataset_final2.shape

(133036, 8)

# Adding the this latest file with the previous data

In [108]:
old_dataset.sort_values(by=['s no'], inplace=True)

In [109]:
old_dataset

,MerchantId,Objectives,IngestionStatus,Report Date,RWM Remarks,Ingestion Date,RWM Revisit Status,RWM Revisit Date,Revisit Expire Date,Validity,Pincode,Phone Number,City,Cluster,Failed Upload RWM MID,csv Google MID,Google csv Visit Date,csv Validity,s no
0,BCR2DN4TSCO3DNDR,checkin-nob,ADD,5-Aug-2022,Success,5-Aug-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,BCR2DN4TWDVZF7LF,checkin-nob,ADD,5-Aug-2022,Merchant Ext.ID is not present in Merchant Mas...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,BCR2DN6T2OKJNTKZ,inactive | stuck_settlements,EDIT,5-Aug-2022,Success,5-Aug-2022,Revisit Done,23-Sep-2022,NaN,VALID,422303.0,9.822789e+09,Nashik,Pune,NaN,BCR2DN6T2OKJNTKZ,23-Sep-2022,VALID,3
3,BCR2DN6TUXLJ3PTX,support | stuck_settlements,EDIT,5-Aug-2022,Success,5-Aug-2022,NaN,NaN,NaN,NaN,160022.0,9.217486e+09,Chandigarh,Delhi,NaN,NaN,NaN,NaN,4
4,BCR2DN6TUW2MV52D,stuck_settlements,ADD,5-Aug-2022,Merchant Ext.ID is not present in Merchant Mas...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991408,BCR2DN6TSWNLLTCJ,Null,DELETE,4-Nov-2022,Success,8-Jun-2022,NaN,NaN,4-Nov-2022,NaN,400024.0,9.833178e+09,Mumbai,Mumbai,NaN,NaN,NaN,NaN,1991409
1991409,BCR2DN4TTSV7H5D7,Inactive - INR 40 Extra Pay on Txn,EDIT,4-Nov-2022,Merchant Ext. ID is not present in the Revisit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1991410
1991410,BCR2DN6TV6F4DTJD,Inactive - INR 40 Extra Pay on Txn,EDIT,4-Nov-2022,Success,8-Jun-2022,NaN,NaN,NaN,NaN,613001.0,9.443183e+09,Trichy,Chennai,NaN,NaN,NaN,NaN,1991411
1991411,BCR2DN6TU6I2LQAC,Inactive - INR 40 Extra Pay on Txn,EDIT,4-Nov-2022,Success,8-Jun-2022,NaN,NaN,NaN,NaN,412803.0,9.850783e+09,Satara,Pune,NaN,NaN,NaN,NaN,1991412


In [110]:
Dynamic_dataset = old_dataset.append(latest_dataset_final2)

In [111]:
Dynamic_dataset.to_csv(output_loc+'Dynamic_source.csv', index=False)